In [6]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import re
import numpy as np
import gzip
import shutil

# help function
def preprocess(x):
    x = str(x) # 轉字串
    x = re.sub('[\u4e00-\u9fa5]','', x) # 去除中文
    x = re.sub(r'[^\w\s]','',x) # 去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 換行符號去除
    x = x.replace('x000D','')
    x = x.replace('_','')
    return str.strip(x) # 移除左右空白

# load data

In [7]:
df = pd.read_excel('../data\寶典\寶典人工處理後\寶典.v8.202111202.xlsx',engine='openpyxl')
df = df.rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})
df['品名'] = df['品名'].apply(lambda x:x.strip())
df

,RIGID,EXTN,公司代號,公司事業部門,品名,memo
0,NaN,NaN,11,塑膠事業部,COPOLYMER PVC C-1250M,NaN
1,NaN,NaN,11,塑膠事業部,COPOLYMER PVC C-15,NaN
2,NaN,NaN,11,塑膠事業部,COPOLYMER PVC C-8,NaN
3,NaN,NaN,11,塑膠事業部,MASS PVC B-57,NaN
4,NaN,NaN,11,塑膠事業部,MASS PVC B-57C,NaN
...,...,...,...,...,...,...
1106,NaN,NaN,61,油品事業部,FUEL OIL,12/2 +
1107,NaN,NaN,6B,油品事業部,JET FUEL,NaN
1108,NaN,NaN,6B,油品事業部,LPG,NaN
1109,NaN,NaN,61,油品事業部,MARINE FUEL OIL,12/2 +


# 載入產品csv

In [8]:
val_df = pd.read_excel('../data/combined_excel.xlsx',index_col=0)[['45A','SPEC','EXPNO','from']].dropna(subset=['45A'],axis=0)
val_df.columns = ['string_X_train','Y_label','EXPNO','from']
val_df

,string_X_train,Y_label,EXPNO,from
0,SHIPMENT OFPVC SUSPENSION RESIN S-65D QTY 18 M...,NaN,NaN,017-202101-03.xlsx
1,"17 MT PVC COPOLYMER RESIN, C-15 AT USD 1470 ...",NaN,NaN,017-202101-03.xlsx
2,"+TERMS OF SALE: CIF NHAVA SHEVA PORT, INDIA++7...",NaN,NaN,017-202101-03.xlsx
3,"MASS PVC RESIN, B-57QUANTITY 175 MT AT 1300 US...",MASS PVC RESIN B-57,11G11F59,017-202101-03.xlsx
4,PHTHALIC ANHYDRIDE (PA)QUANTITY = 306 MT UNIT ...,(鄰苯二甲酐)\nPHTHALIC ANHYDRIDE,27P11346,017-202101-03.xlsx
...,...,...,...,...
20732,PVC SUSPENSION RESIN S65D FOR 70MT AT USD 1270...,NaN,NaN,822-202007-12.xlsx
20733,70.00 MT (IN 4X20 FT CONTAINERS)_x000D_\nPVC R...,PVC RESIN S-60,11G11J49,822-202007-12.xlsx
20734,EPOXY RESIN_x000D_\nNPEF-178_x000D_\nFOB ANY P...,NaN,NaN,822-202007-12.xlsx
20735,EPOXY RESIN_x000D_\nNPEL-128_x000D_\nCIF SAVAN...,NaN,NaN,822-202007-12.xlsx


# 合併寶典和SPEC

In [9]:
產品集合 = set(df['品名'].values.tolist() + val_df['Y_label'].values.tolist())
#產品集合 = set(val_df['Y_label'].values.tolist())
寶典 = [ preprocess(i) for i in 產品集合]
print(len(寶典))
寶典[:5]

2145


['nan',
 'EMULSION PASTES PR1069',
 'RELEASE FILML150A 50UM1300MM4000M',
 'DIESEL FUEL DENSITY 08461',
 'GAG SHEET']

# label萃取

In [10]:
df = pd.read_excel('../data/combined_excel.xlsx',index_col=0)
df = df.rename(columns={'45A':'string_X_train','SPEC':'Y_label'})
df = df[['string_X_train','Y_label','EXPNO','from']].dropna(subset=['string_X_train'],axis=0)
df['from'] = df['from'].apply(lambda x:''.join(x.split('-')[1:]))
df['string_X_train'] = df['string_X_train'].apply(lambda x:preprocess(x))
df['Y_label'] = df['Y_label'].apply(lambda x:str(x).split('\n'))
df['p'] = 'not find'
不認的字 = ['MA','EA','INA','PA','PS']
for idx in tqdm(df.index):
    database = df.loc[idx,'Y_label'] + 寶典
    for p in database:
        if p in df.loc[idx,'string_X_train']:
            if (p != '') & (p != ' ') & (p not in 不認的字):
                df.loc[idx,'p'] = p
                continue

print(df[df['p']!='not find'].shape)
df = df[df['p']!='not find']
df['Y_label'] = df['p']
df = df.drop(['p'],axis=1)

for i in tqdm(df.index):
    x,y = df.loc[i,'string_X_train'],df.loc[i,'Y_label']
    前面的文章 = x[:x.find(y)]
    後面的文章 = x[x.find(y)+len(y):]
    關鍵字 = y
    空白 = ' '
    df.loc[i,'string_X_train'] =  前面的文章 + 空白 + y + 空白 + 後面的文章
print(df.shape)
df.head(10)

C:\Users\ricky\anaconda3\envs\bert_env\lib\site-packages\ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/15273 [00:00<?, ?it/s]

(14267, 5)


C:\Users\ricky\anaconda3\envs\bert_env\lib\site-packages\ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/14267 [00:00<?, ?it/s]

(14267, 4)


,string_X_train,Y_label,EXPNO,from
0,SHIPMENT OFPVC SUSPENSION RESIN S65D QTY 18 MT...,PVC MASS RESIN B57,NaN,20210103.xlsx
1,17 MT PVC COPOLYMER RESIN C15 AT USD 1470 ...,PVC COPOLYMER,NaN,20210103.xlsx
2,TERMS OF SALE CIF NHAVA SHEVA PORT INDIA70 MT ...,PVC RESIN S60,NaN,20210103.xlsx
3,MASS PVC RESIN B57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B57,11G11F59,20210103.xlsx
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27P11346,20210103.xlsx
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2...,LLDPE TAISOX,18Q0C097,20210103.xlsx
6,TERM OF SALE CIF MUNDRA SEAPORT INDIA70 MT OF ...,PVC MASS RESIN B57,NaN,20210103.xlsx
7,COMMODITY ...,POLYESTER PARTIALY ORIENTED YARN,25D11126,20210103.xlsx
8,COMMODITY ...,POLYESTER PARTIALY ORIENTED YARN,25D11126,20210103.xlsx
9,COMMODITY ...,POLYESTER PARTIALY ORIENTED YARN,25D12134,20210103.xlsx


# 起始結束位置標註

In [11]:

def str2index(context,string):
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

ys_lst,ye_lst = [],[]
for i in range(len(df)):
    ys,ye = str2index(df['string_X_train'].values[i],df['Y_label'].values[i])
    ys_lst.append(ys),ye_lst.append(ye)
    
df['string_Y_1'] = ys_lst
df['string_Y_2'] = ye_lst
print(1,df.shape)

df = df[df['Y_label']!=''] # 去掉空值
df = df[df['Y_label']!=' '] # 去掉空值
print(2,df.shape)

df = df.loc[df['string_Y_1'] != -1,:] # 去掉找不到答案的
print(3,df.shape)

df = df.dropna(subset=['string_X_train','Y_label'],axis=0) # 去掉NAN
print(4,df.shape)

df.head(10)

1 (14267, 6)
2 (14267, 6)
3 (14267, 6)
4 (14267, 6)


,string_X_train,Y_label,EXPNO,from,string_Y_1,string_Y_2
0,SHIPMENT OFPVC SUSPENSION RESIN S65D QTY 18 MT...,PVC MASS RESIN B57,NaN,20210103.xlsx,81,99
1,17 MT PVC COPOLYMER RESIN C15 AT USD 1470 ...,PVC COPOLYMER,NaN,20210103.xlsx,8,21
2,TERMS OF SALE CIF NHAVA SHEVA PORT INDIA70 MT ...,PVC RESIN S60,NaN,20210103.xlsx,50,63
3,MASS PVC RESIN B57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B57,11G11F59,20210103.xlsx,1,19
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27P11346,20210103.xlsx,1,19
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2...,LLDPE TAISOX,18Q0C097,20210103.xlsx,11,23
6,TERM OF SALE CIF MUNDRA SEAPORT INDIA70 MT OF ...,PVC MASS RESIN B57,NaN,20210103.xlsx,98,116
7,COMMODITY ...,POLYESTER PARTIALY ORIENTED YARN,25D11126,20210103.xlsx,102,134
8,COMMODITY ...,POLYESTER PARTIALY ORIENTED YARN,25D11126,20210103.xlsx,102,134
9,COMMODITY ...,POLYESTER PARTIALY ORIENTED YARN,25D12134,20210103.xlsx,102,134


# 最後檢查 Y1 Y2 位置 和 Y_LABEL要對應上

In [12]:
for i in tqdm(df.index):
    a = df.loc[i,'string_X_train'][df.loc[i,'string_Y_1']:df.loc[i,'string_Y_2']]
    b = df.loc[i,'Y_label']
    assert a==b

C:\Users\ricky\anaconda3\envs\bert_env\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/14267 [00:00<?, ?it/s]

# most common

In [13]:
df['Y_label'].value_counts().head(20)

PP                      1683
PACK                    1486
RESIN                   1110
PROPYLENE                811
HDPE                     688
SAN                      498
EVA                      420
PHTHALIC ANHYDRIDE       402
GPPS                     373
MONO ETHYLENE GLYCOL     322
EVA TAISOX               297
PE                       273
LAMINATE                 215
HIPS                     212
PLASTIC RESIN            197
FABRICS                  174
ABS                      172
POLYESTER                171
CARBON FIBER TC35        156
CARBON FIBER             153
Name: Y_label, dtype: int64

# 實際感受一下

In [14]:
from termcolor import colored

def str2index(context,string):
    if type(string) != str:
        print(string)
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

def color_output(text_input,text_output):
    ys,ye = str2index(text_input,text_output)
    left = text_input[:ys]
    mid = text_output
    right = text_input[ye:]
    print(left,colored(mid,'red'),right)

for j in range(10):
    i = np.random.randint(low=0,high=len(df))
    x = df['string_X_train'].values[i]
    p = df['Y_label'].values[i]
    print(j)
    color_output(x,p)
    print(colored(p,'green'))

0
1   HDPE   9001   QUANTITY    125 MT   UNIT PRICE  USD1150MT FOB TAIWAN   UNDER FPC CONTRACT H0FX162 HDPE 8010   QUANTITY    100 MT   UNIT PRICE  USD1170MT FOB TAIWAN   UNDER FPC CONTRACT H0FX163 HDPE 8001BL   QUANTITY    50 MT   UNIT PRICE  USD1210MT FOB TAIWAN   UNDER FPC CONTRACT H0FX16OTHER DETAILS ARE AS PER THE APPLICANTS SHIPPINGINSTRUCTION
HDPE
1
PET   RESIN   TRNRTJCTRADE TERMS CFR SHANGHAICHINA
RESIN
2
HS CODE 7410211000 COPPER CLAD LAMINATE QTY206000SHEETS AMOUNT USD80659000HS CODE 3921909090 GLASS EPOXY PREPREG QTY241920PCS AMOUNT USD14268240 TERMS OF PRICE FOB INCOTERMS 2010 OR FCA INCOTERMS 2010 PLACE OF TERMS OF PRICE  BU  SAN   OR INCHEON PORTAIRPORT SOUTH KOREA COUNTRY OF ORIGIN TAIWAN
SAN
3
PRODUCT 1  HIGH DENSITY POLYETHYLENEGRADE  TAISOX 9001QUANTITY  25 MTSUNIT PRICE USD 990 PER MT PER PO 470443301  PACK  ING  25 KG NET IN STANDARD EXPORT PACKINGINCO TERM  FOB ANY PORT IN TAIWANPRODUCT 2  EVAGRADE  TAISOX 7470MQUANTITY  200 MTSUNIT PRICE USD 2280 PER MT PER PO 47

# 保存

In [15]:
print(df.shape)
df.to_csv('../data/preprocess_for_SQUAD_產品.csv')
df

(14267, 6)


,string_X_train,Y_label,EXPNO,from,string_Y_1,string_Y_2
0,SHIPMENT OFPVC SUSPENSION RESIN S65D QTY 18 MT...,PVC MASS RESIN B57,NaN,20210103.xlsx,81,99
1,17 MT PVC COPOLYMER RESIN C15 AT USD 1470 ...,PVC COPOLYMER,NaN,20210103.xlsx,8,21
2,TERMS OF SALE CIF NHAVA SHEVA PORT INDIA70 MT ...,PVC RESIN S60,NaN,20210103.xlsx,50,63
3,MASS PVC RESIN B57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B57,11G11F59,20210103.xlsx,1,19
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27P11346,20210103.xlsx,1,19
...,...,...,...,...,...,...
20732,PVC SUSPENSION RESIN S65D FOR 70MT AT USD 12...,PVC SUSPENSION RESIN S65D,NaN,20200712.xlsx,1,26
20733,7000 MT IN 4X20 FT CONTAINERS PVC RESIN S60 AS...,PVC RESIN S60,11G11J49,20200712.xlsx,30,43
20734,EPOXY RESIN NPEF178FOB ANY PORT OF TAIWAN,RESIN,NaN,20200712.xlsx,7,12
20735,EPOXY RESIN NPEL 128CIF SAVANNAH GA,NPEL,NaN,20200712.xlsx,12,16
